*Copyright (c) Cornac Authors. All rights reserved.*

*Licensed under the Apache 2.0 License.*

# Hyperparameter Search for VAECF

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/PreferredAI/cornac/blob/master/tutorials/param_search_vaecf.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/PreferredAI/cornac/blob/master/tutorials/param_search_vaecf.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This notebook describes how to perform hyperparameter searches in Cornac. As a running example, we consider the VAECF model and MovieLens 100K dataset.

In [1]:
!pip install cornac

## Setup

In [2]:
import numpy as np
import cornac
from cornac.datasets import movielens
from cornac.eval_methods import RatioSplit
from cornac.hyperopt import Discrete, Continuous
from cornac.hyperopt import GridSearch, RandomSearch

c:\Users\bhask\anaconda3\envs\tensorflowgpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare an experiment

First, we load our data and instantiate the necessary objects for running an experiment.

In [3]:
# Load MovieLens 100K ratings
ml_100k = movielens.load_feedback(variant="100K")

# Define an evaluation method to split feedback into train, validation and test sets
ratio_split = RatioSplit(data=ml_100k, test_size=0.1, val_size=0.1, seed=123, verbose=False)

# Instantiate Recall@100 for evaluation
rec100 = cornac.metrics.Recall(100)

# Instantiate VACF with fixed hyperparameters
vaecf = cornac.models.VAECF(k=20, autoencoder_structure=[40], learning_rate=0.005, n_epochs=100, seed=123, verbose=False)

In [18]:
ml_100k

[('196', '242', 3.0),
 ('186', '302', 3.0),
 ('22', '377', 1.0),
 ('244', '51', 2.0),
 ('166', '346', 1.0),
 ('298', '474', 4.0),
 ('115', '265', 2.0),
 ('253', '465', 5.0),
 ('305', '451', 3.0),
 ('6', '86', 3.0),
 ('62', '257', 2.0),
 ('286', '1014', 5.0),
 ('200', '222', 5.0),
 ('210', '40', 3.0),
 ('224', '29', 3.0),
 ('303', '785', 3.0),
 ('122', '387', 5.0),
 ('194', '274', 2.0),
 ('291', '1042', 4.0),
 ('234', '1184', 2.0),
 ('119', '392', 4.0),
 ('167', '486', 4.0),
 ('299', '144', 4.0),
 ('291', '118', 2.0),
 ('308', '1', 4.0),
 ('95', '546', 2.0),
 ('38', '95', 5.0),
 ('102', '768', 2.0),
 ('63', '277', 4.0),
 ('160', '234', 5.0),
 ('50', '246', 3.0),
 ('301', '98', 4.0),
 ('225', '193', 4.0),
 ('290', '88', 4.0),
 ('97', '194', 3.0),
 ('157', '274', 4.0),
 ('181', '1081', 1.0),
 ('278', '603', 5.0),
 ('276', '796', 1.0),
 ('7', '32', 4.0),
 ('10', '16', 4.0),
 ('284', '304', 4.0),
 ('201', '979', 2.0),
 ('276', '564', 3.0),
 ('287', '327', 5.0),
 ('246', '201', 5.0),
 ('242'

## Perform searches for the *beta* parameter

Assume for now we are interested in determining a good value for the hyperparameter $\beta$, the weight of the KL term in VAECF's objective:

$$ \mathcal{L}(\theta,\phi) = \mathbb{E}_{q_{\phi}(z|r)}[\log{p_{\theta}(r|z)}] - \beta \cdot KL(q_{\phi}(z|r)||p(z)), $$

where $z$ is the latent variable (user representation), and $r$ is the observed user-item feedback.

### Wrap vaecf into search methods
All we need to do is to wrap our instantiated model ``vaecf`` into a search method and specify a search space for *beta*, a metric of interest, as well as the evaluation method. Cornac supports two types of searching methods, namely random search and grid search, we consider both of them here.

In [4]:
# GridSearch
gs_vaecf = GridSearch(
    model=vaecf,
    space=[
        Discrete("beta", np.linspace(0.0, 2.0, 11)),
    ],
    metric=rec100,
    eval_method=ratio_split,
)

# RandomSearch
rs_vaecf = RandomSearch(
    model=vaecf,
    space=[
        Continuous("beta", low=0.0, high=2.0),
    ],
    metric=rec100,
    eval_method=ratio_split,
    n_trails=20,
)

As evident from above, there are two types of parameter search domain, namely `Discrete` and `Continuous`. More details in the [documentation](https://cornac.readthedocs.io/en/latest/hyperopt.html).

### Run an experiment
Next, we put everything into an experiment and run it. The results on the validation set, as well as the test results corresponding to the best value of *beta* found by each search method (as measured on the validation set in terms of Recall@100) will be displayed. One can print out more information during this step by setting `verbose=True` when instantiating `VAECF`.

In [5]:
# NDCG = cornac.metrics.NDCG(k=7)
# MAE = cornac.metrics.MAE()
# MSE = cornac.metrics.MSE()
# RMSE = cornac.metrics.RMSE()
# FMeasure = cornac.metrics.FMeasure(k=7)
# map_e = cornac.metrics.MAP()
# auc = cornac.metrics.AUC()
# rec_20 = cornac.metrics.Recall(k=20)

NDCG = cornac.metrics.NDCG()
NDCG_3 = cornac.metrics.NDCG(k=3)
NDCG_4 = cornac.metrics.NDCG(k=4)
NDCG_5 = cornac.metrics.NDCG(k=5)
NDCG_6 = cornac.metrics.NDCG(k=6)
NDCG_7 = cornac.metrics.NDCG(k=7)
NDCG_8 = cornac.metrics.NDCG(k=8)
NDCG_9 = cornac.metrics.NDCG(k=9)
NDCG_10 = cornac.metrics.NDCG(k=10)
NDCG_11 = cornac.metrics.NDCG(k=11)
NDCG_12 = cornac.metrics.NDCG(k=12)
NDCG_13 = cornac.metrics.NDCG(k=13)
NDCG_14 = cornac.metrics.NDCG(k=14)
NDCG_15 = cornac.metrics.NDCG(k=15)
NDCG_16 = cornac.metrics.NDCG(k=16)
NDCG_17 = cornac.metrics.NDCG(k=17)
NDCG_18 = cornac.metrics.NDCG(k=18)
NDCG_19 = cornac.metrics.NDCG(k=19)
NDCG_20 = cornac.metrics.NDCG(k=20)

MRR = cornac.metrics.MRR()
MRR_3 = cornac.metrics.MRR(k=3)
MRR_4 = cornac.metrics.MRR(k=4)
MRR_5 = cornac.metrics.MRR(k=5)
MRR_6 = cornac.metrics.MRR(k=6)
MRR_7 = cornac.metrics.MRR(k=7)
MRR_8 = cornac.metrics.MRR(k=8)
MRR_9 = cornac.metrics.MRR(k=9)
MRR_10 = cornac.metrics.MRR(k=10)
MRR_11 = cornac.metrics.MRR(k=11)
MRR_12 = cornac.metrics.MRR(k=12)
MRR_13 = cornac.metrics.MRR(k=13)
MRR_14 = cornac.metrics.MRR(k=14)
MRR_15 = cornac.metrics.MRR(k=15)
MRR_16 = cornac.metrics.MRR(k=16)
MRR_17 = cornac.metrics.MRR(k=17)
MRR_18 = cornac.metrics.MRR(k=18)
MRR_19 = cornac.metrics.MRR(k=19)
MRR_20 = cornac.metrics.MRR(k=20)

MAE = cornac.metrics.MAE()
MAE_3 = cornac.metrics.MAE(k=3)
MAE_4 = cornac.metrics.MAE(k=4)
MAE_5 = cornac.metrics.MAE(k=5)
MAE_6 = cornac.metrics.MAE(k=6)
MAE_7 = cornac.metrics.MAE(k=7)
MAE_8 = cornac.metrics.MAE(k=8)
MAE_9 = cornac.metrics.MAE(k=9)
MAE_10 = cornac.metrics.MAE(k=10)
MAE_11 = cornac.metrics.MAE(k=11)
MAE_12 = cornac.metrics.MAE(k=12)
MAE_13 = cornac.metrics.MAE(k=13)
MAE_14 = cornac.metrics.MAE(k=14)
MAE_15 = cornac.metrics.MAE(k=15)
MAE_16 = cornac.metrics.MAE(k=16)
MAE_17 = cornac.metrics.MAE(k=17)
MAE_18 = cornac.metrics.MAE(k=18)
MAE_19 = cornac.metrics.MAE(k=19)
MAE_20 = cornac.metrics.MAE(k=20)

MSE = cornac.metrics.MSE()
MSE_3 = cornac.metrics.MSE(k=3)
MSE_4 = cornac.metrics.MSE(k=4)
MSE_5 = cornac.metrics.MSE(k=5)
MSE_6 = cornac.metrics.MSE(k=6)
MSE_7 = cornac.metrics.MSE(k=7)
MSE_8 = cornac.metrics.MSE(k=8)
MSE_9 = cornac.metrics.MSE(k=9)
MSE_10 = cornac.metrics.MSE(k=10)
MSE_11 = cornac.metrics.MSE(k=11)
MSE_12 = cornac.metrics.MSE(k=12)
MSE_13 = cornac.metrics.MSE(k=13)
MSE_14 = cornac.metrics.MSE(k=14)
MSE_15 = cornac.metrics.MSE(k=15)
MSE_16 = cornac.metrics.MSE(k=16)
MSE_17 = cornac.metrics.MSE(k=17)
MSE_18 = cornac.metrics.MSE(k=18)
MSE_19 = cornac.metrics.MSE(k=19)
MSE_20 = cornac.metrics.MSE(k=20)

RMSE = cornac.metrics.RMSE()
RMSE_3 = cornac.metrics.RMSE(k=3)
RMSE_4 = cornac.metrics.RMSE(k=4)
RMSE_5 = cornac.metrics.RMSE(k=5)
RMSE_6 = cornac.metrics.RMSE(k=6)
RMSE_7 = cornac.metrics.RMSE(k=7)
RMSE_8 = cornac.metrics.RMSE(k=8)
RMSE_9 = cornac.metrics.RMSE(k=9)
RMSE_10 = cornac.metrics.RMSE(k=10)
RMSE_11 = cornac.metrics.RMSE(k=11)
RMSE_12 = cornac.metrics.RMSE(k=12)
RMSE_13 = cornac.metrics.RMSE(k=13)
RMSE_14 = cornac.metrics.RMSE(k=14)
RMSE_15 = cornac.metrics.RMSE(k=15)
RMSE_16 = cornac.metrics.RMSE(k=16)
RMSE_17 = cornac.metrics.RMSE(k=17)
RMSE_18 = cornac.metrics.RMSE(k=18)
RMSE_19 = cornac.metrics.RMSE(k=19)
RMSE_20 = cornac.metrics.RMSE(k=20)

R2 = cornac.metrics.R2()
R2_3 = cornac.metrics.R2(k=3)
R2_4 = cornac.metrics.R2(k=4)
R2_5 = cornac.metrics.R2(k=5)
R2_6 = cornac.metrics.R2(k=6)
R2_7 = cornac.metrics.R2(k=7)
R2_8 = cornac.metrics.R2(k=8)
R2_9 = cornac.metrics.R2(k=9)
R2_10 = cornac.metrics.R2(k=10)
R2_11 = cornac.metrics.R2(k=11)
R2_12 = cornac.metrics.R2(k=12)
R2_13 = cornac.metrics.R2(k=13)
R2_14 = cornac.metrics.R2(k=14)
R2_15 = cornac.metrics.R2(k=15)
R2_16 = cornac.metrics.R2(k=16)
R2_17 = cornac.metrics.R2(k=17)
R2_18 = cornac.metrics.R2(k=18)
R2_19 = cornac.metrics.R2(k=19)
R2_20 = cornac.metrics.R2(k=20)

FCP = cornac.metrics.FCP()
FCP_3 = cornac.metrics.FCP(k=3)
FCP_4 = cornac.metrics.FCP(k=4)
FCP_5 = cornac.metrics.FCP(k=5)
FCP_6 = cornac.metrics.FCP(k=6)
FCP_7 = cornac.metrics.FCP(k=7)
FCP_8 = cornac.metrics.FCP(k=8)
FCP_9 = cornac.metrics.FCP(k=9)
FCP_10 = cornac.metrics.FCP(k=10)
FCP_11 = cornac.metrics.FCP(k=11)
FCP_12 = cornac.metrics.FCP(k=12)
FCP_13 = cornac.metrics.FCP(k=13)
FCP_14 = cornac.metrics.FCP(k=14)
FCP_15 = cornac.metrics.FCP(k=15)
FCP_16 = cornac.metrics.FCP(k=16)
FCP_17 = cornac.metrics.FCP(k=17)
FCP_18 = cornac.metrics.FCP(k=18)
FCP_19 = cornac.metrics.FCP(k=19)
FCP_20 = cornac.metrics.FCP(k=20)
# FMeasure = cornac.metrics.FMeasure()
# FMeasure = cornac.metrics.FMeasure(k=7)
# map_e = cornac.metrics.MAP()
# auc = cornac.metrics.AUC()
# rec_20 = cornac.metrics.Recall(k=20)


In [6]:
cornac.Experiment(
    eval_method=ratio_split,
    models=[gs_vaecf, rs_vaecf],
    # metrics=[NDCG, map_e, FMeasure, MAE, MSE, RMSE, rec_20],
    metrics=[NDCG, NDCG_3, NDCG_4, NDCG_5, NDCG_6, NDCG_7, NDCG_8, NDCG_9, NDCG_10, NDCG_11, NDCG_12, NDCG_13, NDCG_14, NDCG_15, NDCG_16, NDCG_17, NDCG_18, NDCG_19, NDCG_20, MAE,MAE_3, MAE_4, MAE_5, MAE_6, MAE_7, MAE_8, MAE_9, MAE_10, MAE_11, MAE_12, MAE_13, MAE_14, MAE_15, MAE_16, MAE_17, MAE_18, MAE_19, MAE_20, MSE, MSE_3, MSE_4, MSE_5, MSE_6, MSE_7, MSE_8, MSE_9, MSE_10, MSE_11, MSE_12, MSE_13, MSE_14, MSE_15, MSE_16, MSE_17, MSE_18, MSE_19, MSE_20, RMSE, RMSE_3, RMSE_4, RMSE_5, RMSE_6, RMSE_7, RMSE_8, RMSE_9, RMSE_10, RMSE_11, RMSE_12, RMSE_13, RMSE_14, RMSE_15, RMSE_16, RMSE_17, RMSE_18, RMSE_19, RMSE_20, R2, R2_3, R2_4, R2_5, R2_6, R2_7, R2_8, R2_9, R2_10, R2_11, R2_12, R2_13, R2_14, R2_15, R2_16, R2_17, R2_18, R2_19, R2_20, MRR, MRR_3, MRR_4, MRR_5, MRR_6, MRR_7, MRR_8, MRR_9, MRR_10, MRR_11, MRR_12, MRR_13, MRR_14, MRR_15, MRR_16, MRR_17, MRR_18, MRR_19, MRR_20, FCP, FCP_3, FCP_4, FCP_5, FCP_6, FCP_7, FCP_8, FCP_9, FCP_10, FCP_11, FCP_12, FCP_13, FCP_14, FCP_15, FCP_16, FCP_17, FCP_18, FCP_19, FCP_20],
    user_based=False,
).run()


VALIDATION:
...
                   | FCP@-1 | FCP@10 | FCP@11 | FCP@12 | FCP@13 | FCP@14 | FCP@15 | FCP@16 | FCP@17 | FCP@18 | FCP@19 | FCP@20 |  FCP@3 |  FCP@4 |  FCP@5 |  FCP@6 |  FCP@7 |  FCP@8 |  FCP@9 | MAE@-1 | MAE@10 | MAE@11 | MAE@12 | MAE@13 | MAE@14 | MAE@15 | MAE@16 | MAE@17 | MAE@18 | MAE@19 | MAE@20 |  MAE@3 |  MAE@4 |  MAE@5 |  MAE@6 |  MAE@7 |  MAE@8 |  MAE@9 | MSE@-1 | MSE@10 | MSE@11 | MSE@12 | MSE@13 | MSE@14 | MSE@15 | MSE@16 | MSE@17 | MSE@18 | MSE@19 | MSE@20 |  MSE@3 |   MSE@4 |   MSE@5 |   MSE@6 |  MSE@7 |  MSE@8 |  MSE@9 |   R2@-1 |    R2@10 |    R2@11 |    R2@12 |   R2@13 |    R2@14 |    R2@15 |    R2@16 |    R2@17 |    R2@18 |    R2@19 |    R2@20 |     R2@3 |     R2@4 |     R2@5 |     R2@6 |     R2@7 |     R2@8 |     R2@9 | RMSE@-1 | RMSE@10 | RMSE@11 | RMSE@12 | RMSE@13 | RMSE@14 | RMSE@15 | RMSE@16 | RMSE@17 | RMSE@18 | RMSE@19 | RMSE@20 | RMSE@3 | RMSE@4 | RMSE@5 | RMSE@6 | RMSE@7 | RMSE@8 | RMSE@9 | MRR@-1 | MRR@10 | MRR@11 | MRR@12 | MRR@13 | MRR@14 | MR

The best *beta* values found by our search methods are as follows,

In [7]:
print('Grid search: beta = {:.2f}'.format(gs_vaecf.best_params.get('beta')))
print('Random search: beta = {:.2f}'.format(rs_vaecf.best_params.get('beta')))

Grid search: beta = 0.80
Random search: beta = 0.85


It is also possible to access the best model through the attribute `best_model`.